**PROJECT MAIN**

In [ ]:
# Data manipulation
import pandas as pd

# Libraries for API querying
import json
import requests
from pprint import pprint

In [ ]:
import os

data_path = 'drive/MyDrive/DSdata'

food_names = []

# Extract food names from folders
for food_name in os.listdir(data_path):
  full_path = os.path.join(data_path, food_name)
  if os.path.isdir(full_path):
    food_names.append(food_name)

print(food_names)

In [ ]:
# Downscale images (ChatGPT)
# Take our raw images
import os
from PIL import Image
from tqdm import tqdm

# Source and target directories
SRC_DIR = "images"        # original images folder with class subfolders
DST_DIR = "food-128"      # output folder

size = (128, 128)  # target resolution

# Create output root directory
os.makedirs(DST_DIR, exist_ok=True)

# Loop through every class folder
for class_name in os.listdir(SRC_DIR):
    src_class_path = os.path.join(SRC_DIR, class_name)
    dst_class_path = os.path.join(DST_DIR, class_name)

    # Skip non-directories (just in case)
    if not os.path.isdir(src_class_path):
        continue

    os.makedirs(dst_class_path, exist_ok=True)

    # Resize each image
    for img_name in tqdm(os.listdir(src_class_path), desc=f"Resizing {class_name[:20]}"):
        src_img_path = os.path.join(src_class_path, img_name)
        dst_img_path = os.path.join(dst_class_path, img_name)

        try:
            img = Image.open(src_img_path).convert("RGB")
            img = img.resize(size, Image.BILINEAR)
            img.save(dst_img_path, quality=90)
        except Exception as e:
            print(f"Skipped {src_img_path}: {e}")

print("All images resized to 128x128 and saved in food-128/")

## FoodData Central API Loading

In [ ]:
# API Setup
BASE_URL = 'https://api.nal.usda.gov/fdc/v1'
API_KEY = 'Ujn2RHBzVpsD6gjTUl83caAgyjbUhEN2HjJcFmn9'

In [ ]:
def search_food(query, data_type='Survey (FNDDS)', limit=5):
    parameters = {
        'api_key': API_KEY,
        'query': query,
        'dataType': data_type,
        'pageSize': limit
    }
    response = requests.get(url=f'{BASE_URL}/foods/search', params=parameters)
    response.raise_for_status()
    return response.json()


def get_calories(food):
    for nutrient in food.get('foodNutrients', []):
        # Test if {foodNutrientId': 34350077, nutrientName: 'Energy'} is the correct dict
        if nutrient.get('nutrientName') == 'Energy' and nutrient.get('unitName') == 'KCAL':
            return nutrient['value']
    return None


def simplify_results(data):
    foods = data.get('foods', [])
    result_foods = []
    for food in foods:
        calories = get_calories(food)
        if calories is not None:
            result_foods.append({
                'name': food['description'],
                'calories': calories
            })
    return result_foods

In [ ]:
try:
    food = input('Type a food:')
    response = search_food(food, limit=1000)
    simplified = simplify_results(response)
    print(len(simplified))
    pprint(simplified)
except Exception as e:
    print('Error:', e)